In [5]:
import real_estate_api as rea

import os
import pickle
import random
import numpy as np
import pandas as pd

from collections import Counter
from itertools import chain

from sklearn.base import TransformerMixin, BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_absolute_error

import keys as k
keys = k.getKeys()

In [2]:
user_home = rea.get_UserHome(keys['SampleHouse'].strip("\'"))
user_home_details = rea.get_PropertyDetail(user_home['property_id'])
hoi = rea.get_HousesOfInterest(user_home, n=2000, listed_to_sold_ratio=0.3, verbose=True)
gd = rea.geo_data(hoi['geo'])

Shortfall in listed houses detected, appending 114 of current listing to results.


In [50]:
# Can this be the first step in the pipeline or no?
fg = rea.FeatureGenerator(
    houses = hoi['houses'], 
    gd=gd,
    user_home=user_home_details
)

In [15]:
# This will randomly take 10% of the data as a validation set.
test_indicies = random.sample(range(0, len(fg.targets)), int(0.2 * len(fg.targets)))
train_indicies = set(range(0, len(fg.targets))).difference(set(test_indicies))

target_transformer = StandardScaler()
train_targets = target_transformer.fit_transform(np.array(list(map(fg.targets.__getitem__, train_indicies))).reshape(-1,1))
test_targets = target_transformer.transform(np.array(list(map(fg.targets.__getitem__, test_indicies))).reshape(-1,1))

train_features = list(map(fg.features.__getitem__, train_indicies))
test_features = list(map(fg.features.__getitem__, test_indicies))

In [41]:
'''
Days Listed - Linear
Days Updated - Linear
*baths - Normalize
year_built - I want to bucketize these, then keep the dummies.
lot_sqft - Normalize
    Feature generation, multiply lot_sqft normalized and the inverse_distance
        More questions, can you do this?
sqft - Normalize
garage / stories / beds - Normalize
Tags - Do what we did in the nlp homework
'''

class ToDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.DataFrame(X).drop_duplicates(subset=['Property_ID'])

class DictEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tags_to_keep = None

    def fit(self, X, y=None):
        tag_frequency = Counter(chain(*X))
        self.tags_to_keep = [k for k, v in tag_frequency.items() if v > 1]
        return self
    
    def transform(self, X):
        return pd.Series(map(lambda l : {k : 1 for k in l if l in self.tags_to_keep}, X))

minmax_cols = ['distance_to_home'] # 'Days_listed', 'Days_listed', 
normalize_cols = ['lot_sqft', 'sqft']
bucketize_cols = ['year_built']
lat_long_cols = ['lat', 'long']
dummy_cols = ['baths_full', 'baths_3qtr', 'baths_half', 'baths_1qtr', 'garage', 'stories', 'beds']

preprocess_min_max_cols = Pipeline(
    [
        ('impute', SimpleImputer(missing_values=np.nan, strategy="mean")),
        ('min_max_scale', MinMaxScaler())
    ]
)

preprocess_tags_col = Pipeline(
    [
        ('dict_encode', DictEncoder()),
        ('dict_vectorize', DictVectorizer())
    ]
)

preprocess_lat_long_cols = Pipeline(
    [
        ('impute', SimpleImputer(missing_values=np.nan, strategy="mean")),
        ('bucketize', KBinsDiscretizer(strategy='uniform'))
    ]
)

preprocess_data = ColumnTransformer(
    [
        #('scale', preprocess_min_max_cols, minmax_cols),
        ('normalize', StandardScaler(), normalize_cols),
        ('bucketize', KBinsDiscretizer(strategy='uniform'), bucketize_cols),
        ('lat_long', preprocess_lat_long_cols, lat_long_cols),
        ('dummy', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), dummy_cols),
        #('list', preprocess_tags_col, 'tags')
    ]
)

In [42]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

class KerasModel(BaseEstimator, RegressorMixin):
    def __init__(self, 
                 user_home, 
                 load_model_if_available : bool = True, 
                 update_model : bool = False, 
                 save_model : bool = False):
        # Interesting note, this instance is created before paramters are passed into the step of the pipeline.
        self.user_home = user_home
        self.load_model_if_available = load_model_if_available
        self.update_model = update_model
        self.save_model = save_model

        self.earlystopping = EarlyStopping(patience=5, verbose=1, min_delta=0.05)

        if update_model and not self.load_model_if_available:
            raise Exception('Can not update a model not loaded.')

        self.model_name = f'{user_home.get("city")}_{user_home.get("state_code")}'

    def _keras_model(self, n_cols):
        km = Sequential()
        km.add(Dense(256, input_shape=(n_cols,), activation='relu', kernel_initializer='normal', name='dense_1'))
        km.add(Dense(128, activation='relu', kernel_initializer='normal', name='dense_2'))
        km.add(Dropout(0.20, name='dropout'))
        km.add(Dense(64, activation='relu', kernel_initializer='normal', name='dense_3'))
        km.add(Dense(1, activation='linear', kernel_initializer='normal', name='output'))
        km.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error'])
        return KerasRegressor(model=km)

    def _generate_model(self, X, y):
            if not self.update_model: 
                self.model = self._keras_model(n_cols=X.shape[1])
            self.model.fit(X, y, epochs=50, batch_size = 50, callbacks = self.earlystopping)

    def fit(self, X, y=None):
        model_file_path = f'Saved Results/KerasModel/{self.model_name}.pkl'

        if self.load_model_if_available:
            if os.path.isfile(model_file_path):
                with open(model_file_path, 'rb') as f:
                    self.model = pickle.load(f)
                if self.update_model:
                    self._generate_model(X, y)
            else:
                print(f'No model found, generating {self.model_name}.')
                self._generate_model(X, y)
        else:
            self._generate_model(X, y)

        if self.update_model or self.save_model:
            with open(model_file_path, 'wb') as f:
                pickle.dump(self.model, f)
        
        return self

    def predict(self, X, tt = target_transformer):
        return tt.inverse_transform(self.model.predict(X))

keras_pipeline = Pipeline(
    [
        ('to_data_frame', ToDataFrame()),
        ('preprocess', preprocess_data),
        ('keras_model', KerasModel(user_home))
    ]
)

keras_pipeline.set_params(**{
    'keras_model__load_model_if_available' : False,
    'keras_model__update_model' : False
})

keras_pipeline.fit(train_features, train_targets)

Epoch 1/50
29/29 [==============================] - 2s 3ms/step - loss: 0.7202 - mean_squared_error: 0.7202 - mean_absolute_error: 0.5823
Epoch 2/50
29/29 [==============================] - 0s 3ms/step - loss: 0.2747 - mean_squared_error: 0.2747 - mean_absolute_error: 0.3630
Epoch 3/50
29/29 [==============================] - 0s 3ms/step - loss: 0.2426 - mean_squared_error: 0.2426 - mean_absolute_error: 0.3385
Epoch 4/50
29/29 [==============================] - 0s 2ms/step - loss: 0.2233 - mean_squared_error: 0.2233 - mean_absolute_error: 0.3250
Epoch 5/50
29/29 [==============================] - 0s 2ms/step - loss: 0.2043 - mean_squared_error: 0.2043 - mean_absolute_error: 0.3150
Epoch 6/50
29/29 [==============================] - 0s 2ms/step - loss: 0.2201 - mean_squared_error: 0.2201 - mean_absolute_error: 0.3235
Epoch 7/50
29/29 [==============================] - 0s 2ms/step - loss: 0.1986 - mean_squared_error: 0.1986 - mean_absolute_error: 0.3093
Epoch 8/50
29/29 [================

Pipeline(steps=[('to_data_frame', ToDataFrame()),
                ('preprocess',
                 ColumnTransformer(transformers=[('normalize', StandardScaler(),
                                                  ['lot_sqft', 'sqft']),
                                                 ('bucketize',
                                                  KBinsDiscretizer(strategy='uniform'),
                                                  ['year_built']),
                                                 ('lat_long',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('bucketize',
                                                                   KBinsDiscretizer(strategy='uniform'))]),
                                                  ['lat', 'long']),
                                                 ('dummy',
                                                  OneHot...
                                       'area_type': 'address',
                                       'centroid': {'lat': 47.572613,
                                                    'lon': -122.306358},
                                       'city': 'Seattle', 'country': 'USA',
                                       'full_address': ['3416 19th Ave S '
                                                        'Seattle WA 98144',
                                                        '3416 19th Ave S '
                                                        'Seattle WA'],
                                       'line': '3416 19th Ave S',
                                       'mpr_id': '1897471296',
                                       'postal_code': '98144',
                                       'prop_status': ['recently_sold',
                                                       'off_market'],
                                       'property_id': '1897471296',
                                       'state_code': 'WA',
                                       'validation_code': ['121']}))])

In [55]:
#best_keras.best_estimator_.score(test_features, target_transformer.inverse_transform(test_targets))
preds = keras_pipeline.predict(test_features)
mean_absolute_error(preds, target_transformer.inverse_transform(test_targets))

11/11 [==============================] - 0s 1ms/step


210790.87625

In [56]:
target_transformer.transform(np.array(fg.user_target).reshape(-1,1))

array([[-0.15784125]])

In [54]:
user_pred = keras_pipeline.predict(fg.user_features)
#target_transformer.inverse_transform(fg.user_target) # I need to figure out how to get this in there.
# From here, I can make a corrective ratio.
# Then get toggles in here to update user_features
user_pred

1/1 [==============================] - 0s 21ms/step


array([[1244988.4]], dtype=float32)

In [52]:
xx = fg.user_features.copy()
xx['baths_half'] += 0
keras_pipeline.predict(xx)

1/1 [==============================] - 0s 24ms/step


array([[1244988.4]], dtype=float32)

In [57]:
fg.user_target

972500